# IMPORTANTE!

Cluster usado para a realização do ETL RAW TO BRONZE deve ser do tipo  single user.
Cluster do tipo "Shered Cluster" vai dar erro no merge caso seja a primeira execução (primeira carga). Esse tipo de cluster faz com que a exeção não seja tratada de forma correta. 


In [0]:
#author: Bruno - Last Update: Victor - 2023-11-07 21:33

from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import lit, when, row_number, current_timestamp, col, trim, coalesce
from pyspark.sql.window import Window
from pyspark.sql import DataFrame
from delta import DeltaTable
import datetime

# Config do spark para não processar lotes que não contenham dados
spark.conf.set("spark.sql.streaming.noDataMicroBatches.enabled", "false")

# Config de evolução de schema na delta table
spark.conf.set("spark.databricks.delta.schema.autoMerge.enabled", "true")

# Variáveis que são definidas no momento de executar o job que estiver ligado a este script
SAVE_LOCATION_PATH = "abfss://dataprep@sgiatec7cloudfivetran.dfs.core.windows.net/checkpoints"

# Variáveis que são definidas no momento de executar o job que estiver ligado a este script
SOURCE_CATALOG_NAME     = dbutils.widgets.get("source_catalog_name") # Nome do catálogo de origem
TARGET_CATALOG_NAME     = dbutils.widgets.get("target_catalog_name") # target catalog name
TAG_PRODUCT             = dbutils.widgets.get("tag_product") # Nome do produto que se encontra na tag do schema
DATABRICKS_SCHEMA_NAME  = dbutils.widgets.get("databricks_schema_name") # target shema name, including db schema
DATA_DICTIONARY         = eval(dbutils.widgets.get("data_dictionary")) # data dictionary table located in the source default schema
IS_UNIFY_DB_PROCESS     = dbutils.widgets.get("is_unify_db_process") # Se "True", vai buscar todos os bancos do mesmo produto, fazer checkpoint em pasta diferente e adicionar coluna _source
UNIFY_TABLE_NAME        = dbutils.widgets.get("unify_table_name")
IGNORE_TABLES = ['fivetran_audit', 'fivetran_log']


In [0]:
class Bronze_layer:
    def __init__(self, source_catalog_name:str, target_catalog_name:str, schema_name:str, table_name:str, pk_dict:dict, fk_list:list, save_location_path:str, merge_condition:str, key_columns_list:list) -> None :
        self.source_catalog_name = source_catalog_name
        self.target_catalog_name = target_catalog_name
        self.schema_name = schema_name
        self.table_name = table_name
        self.pk_dict = pk_dict
        self.fk_list = fk_list
        self.save_location_path = save_location_path
        self.merge_condition = merge_condition
        self.key_columns_list = key_columns_list
    
    def create_table(self, unify_schema=None) -> None:
        """
        Função de criação da tabela de destino na camada bronze utilizando engenharia reversa.
        :return: None
        """

        print(f'{datetime.datetime.now()} - INFO - Creating target table: {self.target_catalog_name}.{self.schema_name}.{self.table_name}')

        # TODO - Criar coluna Primary key "_7cloud", se necessário, que subistituirá a coluna GUID. 
        columns_id_7cloud = '' 

        for key, value in self.pk_dict.items():
            if len(self.pk_dict) == 1:
                if value not in ['int', 'bigint']:
                    columns_id_7cloud += key + '_7cloud BIGINT GENERATED ALWAYS AS IDENTITY (START WITH 1 INCREMENT BY 1),'
        

        # TODO - Coletar os metadados da tabela de origem para criar o schema da tabela de destino
        schema = spark.table(f'{self.source_catalog_name}.{self.schema_name}.{self.table_name}').schema


        # TODO - Limpar os metadados
        lista_colunas = [] 

        for coluna in schema:
            coluna = str(coluna)
            lista_colunas.append(
                coluna[13:].replace("DecimalType", "Decimal")
                                    .replace("',", "")
                                    .replace("Type()", "")
                                    .replace(", True)", "")
                                    .replace(", False)", "")
            )

        lista_colunas.remove('_fivetran_synced Timestamp')
        try:
            lista_colunas.remove('_fivetran_deleted Byte')
        except ValueError:
            lista_colunas.remove('_fivetran_deleted Boolean')

        
        # ------------------------------------------------------------------------------------------------------------
        #                                       Criação Das Colunas Na Tabela Destino
        # ------------------------------------------------------------------------------------------------------------
        
        # TODO - Criar um dicionário contendo os metadados da tabela
        dict_type_columns = dict(subString.lower().split(" ") for subString in lista_colunas)
        
        # TODO - Verificar se é necessário adicionar a coluna _source
        if '_source' in self.key_columns_list:
            self.key_columns_list.remove('_source')
            str_columns = '_source STRING,'
        else:
            str_columns = ''

        # TODO - Unir as colunas Primary Kkey e Foreign Key
        self.key_columns_list = list(set(self.key_columns_list + self.fk_list))

        # TODO - Adicionar colunas chave no início da tabela
        for column in self.key_columns_list:
            str_columns += column + ' ' + dict_type_columns[column] + ','
            dict_type_columns.pop(column)
        
        # TODO - Adicionar colunas "comuns" ao final da tabela
        for key, value in dict_type_columns.items():
            str_columns += key + ' ' + value + ','

        str_columns = str_columns[:-1]

        # TODO - Criar o schema de destino
        if unify_schema is None:
            spark.sql(f'CREATE SCHEMA IF NOT EXISTS {self.target_catalog_name}.{self.schema_name}')
            
            # TODO - Aterar o proprietário do schema para `UnityAdmin`
            spark.sql(f"ALTER SCHEMA {self.target_catalog_name}.{self.schema_name} SET OWNER TO UnityAdmin")
            
            # TODO - Criar a tabela de destino com os metadados da tabela de origem
            spark.sql(f"""
                CREATE TABLE {self.target_catalog_name}.{self.schema_name}.{self.table_name}
                (
                    {columns_id_7cloud}
                    {str_columns}
                )
                TBLPROPERTIES (
                    delta.enableDeletionVectors = true,
                    delta.enableChangeDataFeed = true,
                    delta.autoOptimize.autoCompact=true,
                    delta.autoOptimize.optimizeWrite=true,
                    delta.columnMapping.mode = 'name',
                    delta.minReaderVersion = '3',
                    delta.minWriterVersion = '7',
                    delta.feature.allowColumnDefaults = 'supported'
                )
            """)

            # TODO - Aterar o proprietário da tabela para `UnityAdmin`
            spark.sql(f"ALTER TABLE {self.target_catalog_name}.{self.schema_name}.{self.table_name} SET OWNER TO UnityAdmin")
        else:
            spark.sql(f'CREATE SCHEMA IF NOT EXISTS {self.target_catalog_name}.{unify_schema}')
            # TODO - Aterar o proprietário do schema para `UnityAdmin`
            spark.sql(f"ALTER SCHEMA {self.target_catalog_name}.{unify_schema} SET OWNER TO UnityAdmin")
            
            # TODO - Criar a tabela de destino com os metadados da tabela de origem
            spark.sql(f"""
                CREATE TABLE {self.target_catalog_name}.{unify_schema}.{self.table_name}
                (
                    {columns_id_7cloud}
                    {str_columns}
                )
                TBLPROPERTIES (
                    delta.enableDeletionVectors = true,
                    delta.enableChangeDataFeed = true, 
                    delta.autoOptimize.autoCompact=true,
                    delta.autoOptimize.optimizeWrite=true,
                    delta.columnMapping.mode = 'name',
                    delta.minReaderVersion = '3',
                    delta.minWriterVersion = '7',
                    delta.feature.allowColumnDefaults = 'supported'
                )
            """)

            # TODO - Aterar o proprietário da tabela para `UnityAdmin`
            spark.sql(f"ALTER TABLE {self.target_catalog_name}.{unify_schema}.{self.table_name} SET OWNER TO UnityAdmin")
    
    def erro_except(self, type_error, unify_schema=None) -> None:
        """
        Função para tratamento de erros ocorrido durante o streaming. Os tipos de erros tratados são:
            - CDF: Erro de configuração do Change Data Feed. Ocorre quando a tabela que está sendo estrimada não possui o CDF habilitado.
            - CHECKPOINT: Erro de checkpoint. Ocorre quando o checkpoint não pertence a tabela raw que está sendo processada.
        
        :param type_error: Erro ocorrido durante o streaming.
        :return: None
        """
            
        type_error_str = str(type_error)
        cdf_error = True if 'delta.enableChangeDataFeed=true' in type_error_str else False
        checkpoint_error = True if 'delete your streaming query checkpoint' in type_error_str else False

        if cdf_error:
            print(f'{datetime.datetime.now()} - WARNING - Solving CDF Error')
            
            # TODO - Remover possíveis checkpoints da antiga tabela 
            if unify_schema is None:
                dbutils.fs.rm(f"{self.save_location_path}/{self.source_catalog_name}/{self.schema_name}/{self.table_name}", True)
            else:
                dbutils.fs.rm(f"{self.save_location_path}/unified/{self.source_catalog_name}/{self.schema_name}/{self.table_name}", True)

            # TODO - Habilitar o CDF da tabela com erro
            spark.sql(f"""
                    ALTER TABLE {self.source_catalog_name}.{self.schema_name}.{self.table_name}
                    SET TBLPROPERTIES (
                        delta.enableChangeDataFeed = true,
                        delta.autoOptimize.autoCompact=true,
                        delta.autoOptimize.optimizeWrite=true
                    )
            """)
                
        elif checkpoint_error:
            print(f'{datetime.datetime.now()} - WARNING - Solving Checkpoint Error')
            if unify_schema is None:
                dbutils.fs.rm(f"{self.save_location_path}/{self.source_catalog_name}/{self.schema_name}/{self.table_name}", True)
            
                # Delete Registros que possivelmente estejam na camada Bronze e não foram apagados por causa de ressincronização 
                print(f'{datetime.datetime.now()} - WARNING - Deleting Pendenting Records')
                spark.sql(f"""
                    DELETE FROM {self.target_catalog_name}.{self.schema_name}.{self.table_name} AS target
                    WHERE NOT EXISTS (SELECT * FROM {self.source_catalog_name}.{self.schema_name}.{self.table_name} AS source
                                    WHERE {self.merge_condition})
                """)
            else:
                dbutils.fs.rm(f"{self.save_location_path}/unified/{self.source_catalog_name}/{self.schema_name}/{self.table_name}", True)
            
                # Delete Registros que possivelmente estejam na camada Bronze e não foram apagados por causa de ressincronização 
                print(f'{datetime.datetime.now()} - WARNING - Deleting Pendenting Records')
                spark.sql(f"""
                    DELETE FROM {self.target_catalog_name}.{unify_schema}.{self.table_name} AS target
                    WHERE NOT EXISTS (SELECT * FROM {self.source_catalog_name}.{self.schema_name}.{self.table_name} AS source
                                    WHERE {self.merge_condition[:-36]}) AND target._source = '{self.source_catalog_name}.{self.schema_name}'
                """)

        else:
            raise type_error

    def deduplicate_records(self, batch_df:DataFrame) -> DataFrame:
        """
        Função para deduplicação de registros, onde a última versão do registro será mantida.

        :param batch_df: DataFrame com os registros a serem deduplicados
        :return: DataFrame com os registros deduplicados
        """


        # TODO - Particionar o DataFrame a partir de uma lista de colunas chave para pegar somente a última versão do registro
        window = Window.partitionBy(self.key_columns_list).orderBy(col("_commit_timestamp").desc())
        batch_df = batch_df.withColumn("rank", row_number().over(window)) \
                            .where('rank = 1')
        
        # TODO - Criar uma  coluna com o tipo de operação e remover colunas desnecessárias
        batch_df = batch_df.withColumn('_operation', when(batch_df._fivetran_deleted == 1, lit('D')) \
                                .when(batch_df._fivetran_deleted == True, lit('D')) \
                                .when(batch_df._change_type == 'delete', lit('D')) \
                                .when((batch_df._change_type == 'update_postimage') & (batch_df._fivetran_deleted == 0), lit('U')) \
                                .when((batch_df._change_type == 'update_postimage') & (batch_df._fivetran_deleted == False), lit('U')) \
                                .when(batch_df._change_type == 'insert', lit('I')) \
                                .otherwise('NULL')) \
                            .withColumn('_bronzeUpdateDate', current_timestamp()) \
                            .drop('_fivetran_deleted', '_fivetran_synced', '_change_type', '_commit_version', '_commit_timestamp', 'rank')
        return batch_df

    def merge_changes(self, batch_df:DataFrame, batch_id:int = None, unify_schema:str = None) -> None:
        """
        Função para mesclar mudanças na tabela da camada bronze.
        
        :param batch_df: DataFrame com os registros a serem mesclados
        :param batch_id: ID do batch
        :return: None
        """

        # TODO - Criar um lista de colunas Primary Key para serem utilizadas no particionamento do DataFrame
        for key, value in self.pk_dict.items():
            self.key_columns_list.append(key)
        

        # TODO - Deduplicidade e limpeza do batch
        batch_df = self.deduplicate_records(batch_df)


        # TODO - Tenta ler a tabela de destino. Cria a tabela caso não exista
        if unify_schema is None:
            try:
                delta_df = DeltaTable.forName(spark, f'{self.target_catalog_name}.{self.schema_name}.{self.table_name}')
            except Exception as e:
                self.create_table()
                delta_df = DeltaTable.forName(spark, f'{self.target_catalog_name}.{self.schema_name}.{self.table_name}')
        else:
            try:
                delta_df = DeltaTable.forName(spark, f'{self.target_catalog_name}.{unify_schema}.{self.table_name}')
            except Exception as e:
                self.create_table(unify_schema=unify_schema)
                delta_df = DeltaTable.forName(spark, f'{self.target_catalog_name}.{unify_schema}.{self.table_name}')
        

        # TODO - Execute o MERGE com Evolution Schema
        delta_df.alias("target") \
            .merge(batch_df.alias("source"), self.merge_condition) \
            .whenMatchedDelete("source._operation = 'D'") \
            .whenMatchedUpdateAll() \
            .whenNotMatchedInsertAll("source._operation != 'D'") \
            .execute()

    def streaming_table(self) -> None:
        """
        Função para streaming da tabela usando o Change Data Feed. 
        Esse streaming chama uma função que resultará em um merge das atualizações da tabela da camada bronze.
        
        :return: None
        """

        (
            spark.readStream
                .option("readChangeData", "true")
                .table(f"{self.source_catalog_name}.{self.schema_name}.{self.table_name}")
                .filter(col('_change_type').isin(["insert","delete","update_postimage"]))
            .writeStream
                .foreachBatch(self.merge_changes)
                .option("checkpointLocation", f"{self.save_location_path}/{self.source_catalog_name}/{self.schema_name}/{self.table_name}/_checkpoint")
                .trigger(availableNow=True)
            .start()
            .awaitTermination()
        )
    
    def streaming_table_append_only(self, tag_product:str) -> None:
        """
        Função para streaming em modo "append only" da tabela usando o Change Data Feed. O destino desse streaming é uma tabela de stage que conterá vários appends, correspondendo a atualizações de tabelas que serão unificadas na camada bronze.

        :param tag_product: Tag do produto que servirá para identificar a tabela de destino.
        :return: None
        """

        (
            spark.readStream
                .option("readChangeData", "true")
                .table(f"{self.source_catalog_name}.{self.schema_name}.{self.table_name}")
                .filter(col('_change_type').isin(["insert","delete","update_postimage"]))
                .withColumn('_source', lit(f'{self.source_catalog_name}.{self.schema_name}')) \
            .writeStream
                .option("checkpointLocation", f"{self.save_location_path}/unified/{self.source_catalog_name}/{self.schema_name}/{self.table_name}/_checkpoint")
                .trigger(availableNow=True)
                .table(f"{self.target_catalog_name}.stage.{tag_product}_{self.table_name}")
            .awaitTermination()
        )


In [0]:
def get_primary_key(source_catalog_name:str, table_name:str, data_dictionary:dict) -> dict:
    """
    Função para pegar a(s) Primary Key(s) da tabela e seu tipo de dado usando um dicionário de dados.
    Ex: {'id': 'bigint'}

    :param source_catalog_name: Nome do catálogo da origem
    :param table_name: Nome da tabela
    :param data_dictionary: Dicionário com as informações do Data Dictionary da tabela
    :return: Dicionário com a Primary Key
    """
    
    pk_df = spark.sql(f"""
        SELECT ColunaNome, ColunaTipo
        FROM {source_catalog_name}.default.{data_dictionary["dict_name"]}
        WHERE DatabaseName = '{data_dictionary["database"]}' 
            AND SchemaNome = '{data_dictionary["schema"]}' 
            AND TabelaNome = '{table_name}' 
            AND IsPK = 'PK'
    """)

    pk_dict = {row['ColunaNome']: row['ColunaTipo'] for row in pk_df.collect()}
    return pk_dict

def get_foreing_key(source_catalog_name:str, table_name:str, data_dictionary:dict) -> list:
    """
    Função para pegar a(s) Foreign Key(s) da tabela e seu tipo de dado usando um dicionário de dados.
    Ex: ['fk_table_name']

    :param source_catalog_name: Nome do catálogo da origem
    :param table_name: Nome da tabela
    :param data_dictionary: Dicionário com as informações do Data Dictionary da tabela
    :return: Lista com as Foreign Keys
    """

    fk_table = spark.sql(f"""
        SELECT ColunaNome
        FROM {source_catalog_name}.default.{data_dictionary["dict_name"]}
        WHERE DatabaseName = '{data_dictionary["database"]}' 
            AND SchemaNome = '{data_dictionary["schema"]}' 
            AND TabelaNome = '{table_name}' 
            AND trim(FKRef) != ''
            AND coalesce(IsPK, '0') = '0'
    """)

    fk_list = [row['ColunaNome'] for row in fk_table.collect()]
    return fk_list
    

In [0]:
def tables_from_schema(schema_name:str, source_catalog_name:str) -> list:
    """
    Função para pegar uma lista com as tabelas que estão no schema informado.

    :param schema_name: Nome do schema
    :param source_catalog_name: Nome do catálogo da origem
    :return: Lista com as tabelas
    """

    table_name_df = spark.sql(f"""
        SELECT table_name
        FROM {source_catalog_name}.information_schema.tables
        WHERE table_schema = '{schema_name}'
        ORDER BY table_name
    """)

    table_list = [row['table_name'] for row in table_name_df.collect()]
    return table_list

In [0]:
def schemas_from_table(table_name:str, source_catalog_name:str, tag_product:str) ->list:
    """
    Função para pegar uma lista com os schemas que contém a tabela informada.

    :param table_name: Nome da tabela
    :param source_catalog_name: Nome do catálogo da origem
    :param tag_product: Nome do tag do produto
    :return: Lista com os schemas
    """
    schemas_df = spark.sql(f"""
        SELECT      table_schema
        FROM        {source_catalog_name}.information_schema.tables t
        INNER JOIN  {source_catalog_name}.information_schema.schema_tags st     ON st.schema_name = t.table_schema
        WHERE       t.table_name = '{table_name}' AND st.tag_name = 'product'   AND st.tag_value = '{tag_product}'
        ORDER BY    table_schema     ASC
    """)

    schemas_list = [row['table_schema'] for row in schemas_df.collect()]
    return schemas_list

In [0]:
def unified_table_process(unify_table_name:str, source_catalog_name:str, target_catalog_name:str, tag_product:str, unify_schema:str, data_dictionary:dict, ignore_tables:str, save_location_path:str) -> None:
    """
    Função para iniciar o processo atualizações de tabelas unificadas.

    :param unify_table_name: Nome da tabela unificada
    :param source_catalog_name: Nome do catálogo da origem
    :param target_catalog_name: Nome do catálogo de destino
    :param tag_product: Nome do tag do produto
    :param data_dictionary: Dicionário com as informações do data dictionary
    :param ignore_tables: Lista com as tabelas que não devem ser enviadas para a camada bronze
    :param save_location_path: Caminho onse serão salvos os checkponts
    :return: None
    """

    # TODO - Criar o schema stage para armazenar as tabelas de stage
    spark.sql(f"CREATE SCHEMA IF NOT EXISTS {target_catalog_name}.stage")
    spark.sql(f"ALTER SCHEMA {target_catalog_name}.stage SET OWNER TO UnityAdmin")

    # TODO - Pegar as primary keys e foreign keys da tabela
    pk_dict = get_primary_key(source_catalog_name=source_catalog_name, table_name=unify_table_name, data_dictionary=data_dictionary)
    fk_list = get_foreing_key(source_catalog_name=source_catalog_name, table_name=unify_table_name, data_dictionary=data_dictionary)

    # TODO - Criar uma variável para armazenar a condição de merge
    merge_condition = ''
    for key, value in pk_dict.items():
        merge_condition += 'target.' + key + ' = source.' + key + ' AND '
    merge_condition = merge_condition + "target._source = source._source"

    # TODO - Pegar uma lista de schemas que contem a tabela
    schemas_list = schemas_from_table(unify_table_name, source_catalog_name, tag_product)

    # TODO - Iniciar o processo de captura de atualizações de registros
    for schema_name in schemas_list:
        print(f'{datetime.datetime.now()} - INFO - Starting Stream: {source_catalog_name}.{schema_name}.{unify_table_name}')

        table_obj = Bronze_layer(
            source_catalog_name = source_catalog_name,
            target_catalog_name = target_catalog_name,
            table_name = unify_table_name,
            schema_name = schema_name,
            pk_dict = pk_dict,
            fk_list = fk_list,
            save_location_path = save_location_path,
            merge_condition = merge_condition,
            key_columns_list = ['_source']
        )

        try:
            table_obj.streaming_table_append_only(tag_product=tag_product)
        except Exception as type_error:
            table_obj.erro_except(type_error=type_error, unify_schema=unify_schema)
            print(f'{datetime.datetime.now()} - WARNING - Restarting streaming table')
            table_obj.streaming_table_append_only(tag_product=tag_product)

        print(f'{datetime.datetime.now()} - INFO - Finished Stream: {source_catalog_name}.{schema_name}.{unify_table_name}')


        # TODO - Iniciar o processo de merge de registros caso seja o último schema
        if schema_name == schemas_list[-1]:
            unify_stage_df = spark.table(f"{table_obj.target_catalog_name}.stage.{tag_product}_{table_obj.table_name}")

            table_obj.merge_changes(batch_df=unify_stage_df, unify_schema=unify_schema)

            # TODO - Apagar a tabela de stage no fim do processo.
            spark.sql(f"DROP TABLE IF EXISTS {table_obj.target_catalog_name}.stage.{tag_product}_{table_obj.table_name}") 

    
    # TODO - Verificar a integridade das tabelas unificadas
    print(f'{datetime.datetime.now()} - INFO - Start Integrity Verification: {unify_schema}_{unify_table_name}')
    linha_integridade = []


    for schema_name in schemas_list:
        # TODO - Realizar a contagem de quantos registros existem na tabela raw e na tabela bronze.
        # Caso falhe, o processo irá registrar 0 (zero) para cada uma das colunas
        try:
            qtd_raw = spark.sql(f"""
                SELECT * FROM {source_catalog_name}.`{schema_name}`.`{unify_table_name}`
                WHERE _fivetran_deleted = False OR _fivetran_deleted = 0
            """).count()                                                                                                            

            qtd_bronze = spark.sql(f"""
                SELECT * FROM {target_catalog_name}.`{unify_schema}`.`{unify_table_name}`
                WHERE _source = '{source_catalog_name}.{schema_name}'
            """).count()

            linha_integridade.append((schema_name, unify_table_name, qtd_raw, qtd_bronze))
        except:
            linha_integridade.append((schema_name, unify_table_name, 0, 0))

    # TODO - Salvar a integridade das tabelas unificadas
    schema_integridade = StructType([
                    StructField('nome_schema', StringType(), True),
                    StructField('nome_table', StringType(), True),
                    StructField('qtd_raw', IntegerType(), True),
                    StructField('qtd_bronze', IntegerType(), True)
                    ])

    df_integridade = spark.createDataFrame(linha_integridade, schema_integridade)
    df_integridade.write.mode('overwrite').saveAsTable(f'{target_catalog_name}.default.integridade_{unify_table_name}')

    print(f'{datetime.datetime.now()} - INFO - End of Integrity Verification: {unify_schema}_{unify_table_name}')


In [0]:
def schema_table_process(source_catalog_name:str, target_catalog_name:str, databricks_schema_name:str, data_dictionary:dict, ignore_tables:str, save_location_path:str) -> None:
    """
    Função para iniciar o processo de ETL de tabelas que estão dentro de um schema específico.
    
        :param source_catalog_name: Nome do catálogo da origem
        :param target_catalog_name: Nome do catálogo de destino
        :param databricks_schema_name: Nome do schema do Databricks que será processado
        :param data_dictionary: Dicionário com as informações do dicionário de dados das tabelas do produto
        :param ignore_tables: Lista com as tabelas que não devem ser enviadas para a camada bronze, como tabelas de controle do Fivetran. 
            Ex: ['fivetran_connection', 'fivetran_log']
        :param save_location_path: Caminho onde serão salvos os checkponts
        :return: None
    """

    # Usar o catálogo fonte que foi passado como parâmetro no script
    table_list = tables_from_schema(schema_name=databricks_schema_name, source_catalog_name=source_catalog_name)
    linha_integridade = []

    # Laço de repetição para iniciar o streaming das tabelas que serão enviadas para a camada bronze
    for table in table_list:

        # Condição se a tabela deve ou não ser enviada para a camada bronze
        # EX: Não enviar tabelas de controle do Fivetran (tabela fivetran_audit)
        if table not in ignore_tables:
            # TODO - Pegar as primary keys e foreign keys da tabela
            pk_dict = get_primary_key(source_catalog_name=source_catalog_name, table_name=table, data_dictionary=data_dictionary)
            fk_list = get_foreing_key(source_catalog_name=source_catalog_name, table_name=table, data_dictionary=data_dictionary)

            # TODO - Criar uma variável para armazenar a condição de merge
            merge_condition = ''
            for key, value in pk_dict.items():
                merge_condition += 'target.' + key + ' = source.' + key + ' AND '
            merge_condition = merge_condition[:-5]

                
            # TODO - Criar o obejeto tabela que será usada no streaming
            table_obj = Bronze_layer(
                source_catalog_name=source_catalog_name,
                target_catalog_name=target_catalog_name,
                schema_name=databricks_schema_name,
                table_name=table,
                pk_dict=pk_dict,
                fk_list=fk_list,
                save_location_path=save_location_path,
                merge_condition=merge_condition,
                key_columns_list=[]
            )

            # TODO - Iniciar o streaming do objeto. Caso der erro, resolver as exceções e reiniciar o processo de streaming
            try:
                print(f'{datetime.datetime.now()} - INFO - Starting Stream: {source_catalog_name}.{databricks_schema_name}.{table}')
                table_obj.streaming_table()
                
                # TODO - Realizar a contagem de quantos registros existem na tabela raw e na tabela bronze.
                # Caso falhe, o processo irá registrar 0 (zero) para cada uma das colunas
                """
                Nota: o processo de ferificação de integridade foi colocado nesse ponto para ter uma menor latência entre a chegada de dados na camada raw e o ETL para a camada bronze. Dessa forma, a checagem de integridade será realizada logo após a passagem dos dados de cada tabela para a camada bronze, não sendo necessário esperar o fim do ETL de todas as tabelas, como ocorre nas tabelas unificadas. 
                """
                try:
                    qtd_bronze = spark.table(f'{target_catalog_name}.`{databricks_schema_name}`.`{table}`').count()
                    qtd_raw = spark.table(f'{source_catalog_name}.`{databricks_schema_name}`.`{table}`') \
                                    .filter((col('_fivetran_deleted') == False) | (col('_fivetran_deleted') == 0)).count()
                                    
                    linha_integridade.append((databricks_schema_name, table, qtd_raw, qtd_bronze))
                except:
                    linha_integridade.append((databricks_schema_name, table, 0, 0))

            except Exception as type_error:
                table_obj.erro_except(type_error=type_error)
                print(f'{datetime.datetime.now()} - WARNING - Restarting streaming table')
                table_obj.streaming_table()
            
            print(f'{datetime.datetime.now()} - INFO - Finished Stream: {source_catalog_name}.{databricks_schema_name}.{table}')


    # TODO - Salvar tabela de integridade em modo overwrite
    schema_integridade = StructType([
                    StructField('nome_schema', StringType(), True),
                    StructField('nome_table', StringType(), True),
                    StructField('qtd_raw', IntegerType(), True),
                    StructField('qtd_bronze', IntegerType(), True)
                    ])

    df_integridade = spark.createDataFrame(linha_integridade, schema_integridade)
    df_integridade.write.mode('overwrite').saveAsTable(f'{target_catalog_name}.default.integridade_{databricks_schema_name}')

In [0]:
if IS_UNIFY_DB_PROCESS == "True":
    unified_table_process(
        unify_table_name        = UNIFY_TABLE_NAME,
        source_catalog_name     = SOURCE_CATALOG_NAME,
        target_catalog_name     = TARGET_CATALOG_NAME,
        tag_product             = TAG_PRODUCT,
        unify_schema            = DATABRICKS_SCHEMA_NAME,
        data_dictionary         = DATA_DICTIONARY,
        ignore_tables           = IGNORE_TABLES,
        save_location_path      = SAVE_LOCATION_PATH
    )
elif IS_UNIFY_DB_PROCESS == "False":
    schema_table_process(
        source_catalog_name     = SOURCE_CATALOG_NAME,
        target_catalog_name     = TARGET_CATALOG_NAME,
        databricks_schema_name  = DATABRICKS_SCHEMA_NAME,
        data_dictionary         = DATA_DICTIONARY,
        ignore_tables           = IGNORE_TABLES,
        save_location_path      = SAVE_LOCATION_PATH
    )
else:
    assert False, "A variável IS_UNIFY_DB_PROCESS deve ser 'True' ou 'False'."